#  Calcul formel pour inférience bayésienne de processus de Poisson pour extremes


Il existe une librairie `stats` de Scipy pour les extrêmes la possibilité d'évaluer la cdf, pdf, etc., et même une fonction "fit" par MLE.

*Attention cependant* : `genextreme.pdf(x, c, loc, scale)` = $f_{GEV}(x|\mu, \sigma, -\xi)$  

In [2]:
import numpy as np
from sympy import *
from sympy.stats import *
import matplotlib.pyplot as plt

# Processus de Poisson reparamétrisé

In [3]:
r, a, gam = symbols("r alpha gamma")
b = symbols("nu")

m, u, n = symbols("m u n")

x = IndexedBase("x")
i = Symbol("i")


#likelihood_PP = -r + n*log(r/m) -n*log(gam*(u-a)) \
#                - (1/gam + 1)*Sum(log((x[i] - a)/(u - a)), (i, 1, n))


likelihood_PP = -r + n*log(1+gam) -n*log(b) + n*log(r/m) \
                - (1/gam + 1)*Sum(log(1+(gam*(1+gam)/b)*(x[i] - u)), (i, 1, n))


likelihood_PP

-n*log(nu) + n*log(r/m) + n*log(gamma + 1) - r - (1 + 1/gamma)*Sum(log(gamma*(gamma + 1)*(-u + x[i])/nu + 1), (i, 1, n))

In [4]:
observed_inf = [diff(likelihood_PP, r, r), diff(likelihood_PP, r, b), diff(likelihood_PP, r, gam), 
                diff(likelihood_PP, b, b), diff(likelihood_PP, b, gam),
                diff(likelihood_PP, gam, gam)]

observed_fish = [0, 0, 0, 0, 0, 0]

Pour la suite on pose $v = \frac{u-\mu}{\sigma}$ et $z_i = \frac{x_i-\mu}{\sigma}$ 

In [5]:
# !pip install antlr4-python3-runtime==4.7.1

from sympy.parsing.latex import parse_latex

# Test : 
parse_latex(r"\frac{1}{x}")

v = symbols("v")

z = IndexedBase("z")

## Calcul de $\frac{\partial^2 l}{\partial\mu^2}$ :

In [6]:
observed_inf[0]

-n/r**2

## Calcul de $\frac{\partial^2 l}{\partial\mu\partial\sigma}$ :

In [7]:
observed_inf[1]

0

### Calcul de $\frac{\partial^2 l}{\partial\mu\partial\gamma}$ :

In [8]:
observed_inf[2]

0

### Calcul de $\frac{\partial^2 l}{\partial\sigma^2}$ :

In [9]:
observed_inf[3]

(-gamma*(1 + 1/gamma)*(gamma + 1)*Sum((u - x[i])*(-gamma*(gamma + 1)*(u - x[i])/(nu*(gamma*(gamma + 1)*(u - x[i])/nu - 1)) + 2)/(gamma*(gamma + 1)*(u - x[i])/nu - 1), (i, 1, n))/nu + n)/nu**2

### Calcul de $\frac{\partial^2 l}{\partial\sigma\partial\gamma}$ :

In [10]:
observed_inf[4]

-((1 + 1/gamma)*(2*gamma + 1)*Sum((u - x[i])*(gamma*(gamma + 1)*(u - x[i])/(nu*(gamma*(gamma + 1)*(u - x[i])/nu - 1)) - 1)/(gamma*(gamma + 1)*(u - x[i])/nu - 1), (i, 1, n)) + (gamma + 1)*Sum((u - x[i])/(gamma*(gamma + 1)*(u - x[i])/nu - 1), (i, 1, n))/gamma)/nu**2

### Calcul de $\frac{\partial^2 l}{\partial\gamma^2}$ :

In [11]:
observed_inf[5]

-n/(gamma + 1)**2 - (1 + 1/gamma)*Sum((2 - (2*gamma + 1)**2*(u - x[i])/(nu*(gamma*(gamma + 1)*(u - x[i])/nu - 1)))*(u - x[i])/(gamma*(gamma + 1)*(u - x[i])/nu - 1), (i, 1, n))/nu + 2*(2*gamma + 1)*Sum((u - x[i])/(gamma*(gamma + 1)*(u - x[i])/nu - 1), (i, 1, n))/(gamma**2*nu) - 2*Sum(log(-gamma*(gamma + 1)*(u - x[i])/nu + 1), (i, 1, n))/gamma**3